In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
## Langmith tracking
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")

In [ ]:
## Pdf reader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('attention.pdf')
docs=loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
documents=text_splitter.split_documents(docs)
documents[:5]

In [ ]:
## FAISS Vector Database
from langchain_openai import OpenAIEmbeddings
# from langchain_community.vectorstores import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db = FAISS.from_documents(documents[:30], OpenAIEmbeddings())

In [ ]:
db

In [ ]:
## query from FAISS Vector DB
query = "Who are the authors of attention is all you need?"
retireved_results=db.similarity_search(query)
print(retireved_results[0].page_content)

In [ ]:
# from langchain_community.llms import Ollama
# llm=Ollama(model="llama2")

from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-3.5-turbo")


In [ ]:
llm

In [ ]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer.
I will tip you $1000 if the user finds the answer helpful.
<context>
{context}
</context>
Question: {input}"""
)


In [ ]:
## Chain Introduction
## Create Stuff Document Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)


In [ ]:
"""A retriever is an interface that returns documents given an unstructured query. 
It is more general than a vector store. A retriever does not need to be able to store documents,
only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, 
but there are other types of retrievers as well.
"""

retriever=db.as_retriever()
retriever

In [ ]:
from langchain.chains import create_retrieval_chain

retriever_chain  = create_retrieval_chain(retriever, document_chain)

In [ ]:
response=retriever_chain.invoke({"input": "An attention function can be described as mapping a query"})

In [ ]:
response['answer']